<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Recomendação Top N

Este notebook contém 2 exemplos de recomendação Top N:

- **Top N consumidos**: os N itens mais consumidos pelos usuários
- **Top N avaliados**: os N itens melhores avaliados pelos usuários

O dataset a ser utilizado será o [MovieLens](https://grouplens.org/datasets/movielens/), cuja análise exploratória foi feita no exemplo prático do módulo **Introdução aos Sistemas de Recomendação**.

In [1]:
import os
import sys
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

# Carregando e processando o dataset

Para mais informações desta sessão, consulte o notebook `Análise Exploratória do MovieLens` do módulo 01.

## Arquivo de avaliações

Upload file `ratings.parquet`

In [2]:
%%time
_ = files.upload() # approx: 1min56s

Saving ratings.parquet to ratings.parquet
CPU times: user 1.74 s, sys: 306 ms, total: 2.05 s
Wall time: 1min 55s


In [3]:
df_ratings = pd.read_parquet('ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [4]:
%%time
_ = files.upload() # approx: 10s

Saving movies.parquet to movies.parquet
CPU times: user 1.08 s, sys: 132 ms, total: 1.21 s
Wall time: 1min 16s


In [5]:
def convert_genres_to_list(genres:str, separator='|'):
    return genres.split(separator)

df_items = pd.read_parquet('movies.parquet')
df_items['genres'] = df_items['genres'].apply(convert_genres_to_list)
df_items.tail()

,item_id,title,genres
3878,3948,Meet the Parents (2000),[Comedy]
3879,3949,Requiem for a Dream (2000),[Drama]
3880,3950,Tigerland (2000),[Drama]
3881,3951,Two Family House (2000),[Drama]
3882,3952,"Contender, The (2000)","[Drama, Thriller]"


# Recomendação Top N Consumidos

Em nosso primeiro exemplo de recomendação não-personalizada iremos recomendar os items mais consumidos pelos usuários.

Em geral, uma função de recomendação retorna 2 tipos de informação:

- `item_id`: identificador do item
- `score`: _score_ a ser utilizado para ordenação da oferta para o usuário

In [13]:
def recommend_top_n_consumptions(ratings:pd.DataFrame, n:int) -> pd.DataFrame:

    recommendations = (
            ratings
            .groupby('item_id')
            .count()['user_id']
            .reset_index()
            .rename({'user_id': 'score'}, axis=1)
            .sort_values(by='score', ascending=False)
    )

    return recommendations.head(n)

df_top_consumptions = recommend_top_n_consumptions(df_ratings, n=10)
df_top_consumptions

,item_id,score
2651,2858,3428
253,260,2991
1106,1196,2990
1120,1210,2883
466,480,2672
1848,2028,2653
575,589,2649
2374,2571,2590
1178,1270,2583
579,593,2578
